#Siamese Networks:
Siamese networks are designed to learn similarity between two inputs. They consist of two identical subnetworks (or branches) with shared weights. These networks are used for tasks like face verification, signature verification, and similarity-based recommendation systems.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
import numpy as np

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Define the Siamese network architecture
def create_siamese_network(input_shape):
    # Define the base network (shared subnetwork)
    input = layers.Input(shape=input_shape)
    x = layers.Flatten()(input)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(128, activation='relu')(x)
    base_network = models.Model(input, x)

    # Define the two input branches of the Siamese network
    input_a = layers.Input(shape=input_shape)
    input_b = layers.Input(shape=input_shape)

    # Connect each input to the base network
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)

    # Calculate L1 distance between the processed inputs
    distance = tf.reduce_sum(tf.abs(processed_a - processed_b), axis=1, keepdims=True)

    # Create the Siamese model
    siamese_model = models.Model([input_a, input_b], distance)

    return siamese_model

# Instantiate the Siamese network
input_shape = x_train.shape[1:]
siamese_model = create_siamese_network(input_shape)

# Compile the model
siamese_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

def create_pairs(x, y, num_classes):
    pairs, labels = [], []
    class_indices = [np.where(y == i)[0] for i in range(num_classes)]

    for class_idx in range(num_classes):
        for i in range(len(class_indices[class_idx])):
            x1 = x[class_indices[class_idx][i]]
            label1 = class_idx

            # Create a positive pair (same class)
            idx2 = i + 1 if i + 1 < len(class_indices[class_idx]) else 0
            x2 = x[class_indices[class_idx][idx2]]
            pairs.append([x1, x2])
            labels.append(1)

            # Create a negative pair (different class)
            other_class_idx = (class_idx + 1) % num_classes
            other_class_indices = class_indices[other_class_idx]
            random_idx = np.random.choice(len(other_class_indices))
            x2 = x[other_class_indices[random_idx]]
            pairs.append([x1, x2])
            labels.append(0)

    return np.array(pairs), np.array(labels)

# Create pairs for training and validation
num_classes = len(np.unique(y_train))
train_pairs, train_labels = create_pairs(x_train, y_train, num_classes)
test_pairs, test_labels = create_pairs(x_test, y_test, num_classes)

# Train the Siamese network
siamese_model.fit([train_pairs[:, 0], train_pairs[:, 1]], train_labels,
                  validation_data=([test_pairs[:, 0], test_pairs[:, 1]], test_labels),
                  batch_size=128,
                  epochs=20)

Epoch 1/20
938/938 [==============================] - 10s 9ms/step - loss: 7.6246 - accuracy: 0.5000 - val_loss: 7.6246 - val_accuracy: 0.5000
Epoch 2/20
938/938 [==============================] - 7s 8ms/step - loss: 7.6246 - accuracy: 0.5000 - val_loss: 7.6246 - val_accuracy: 0.5000
Epoch 3/20
938/938 [==============================] - 6s 7ms/step - loss: 7.6246 - accuracy: 0.5000 - val_loss: 7.6246 - val_accuracy: 0.5000
Epoch 4/20
938/938 [==============================] - 7s 8ms/step - loss: 7.6246 - accuracy: 0.5000 - val_loss: 7.6246 - val_accuracy: 0.5000
Epoch 5/20
938/938 [==============================] - 6s 7ms/step - loss: 7.6246 - accuracy: 0.5000 - val_loss: 7.6246 - val_accuracy: 0.5000
Epoch 6/20
938/938 [==============================] - 7s 8ms/step - loss: 7.6246 - accuracy: 0.5000 - val_loss: 7.6246 - val_accuracy: 0.5000
Epoch 7/20
938/938 [==============================] - 6s 7ms/step - loss: 7.6246 - accuracy: 0.5000 - val_loss: 7.6246 - val_accuracy: 0.5000
Epoch

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Define the Siamese network architecture
def create_siamese_network(input_shape):
    # Define the base network (shared subnetwork)
    input = layers.Input(shape=input_shape)
    x = layers.Flatten()(input)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(128, activation='relu')(x)
    base_network = models.Model(input, x)

    # Define the two input branches of the Siamese network
    input_a = layers.Input(shape=input_shape)
    input_b = layers.Input(shape=input_shape)

    # Connect each input to the base network
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)

    # Calculate L1 distance between the processed inputs
    distance = tf.reduce_sum(tf.abs(processed_a - processed_b), axis=1, keepdims=True)

    # Create the Siamese model
    siamese_model = models.Model([input_a, input_b], distance)

    return siamese_model

# Create pairs with data augmentation for training
def create_pairs_with_augmentation(x, y, num_classes, num_pairs_per_class):
    pairs, labels = [], []
    class_indices = [np.where(y == i)[0] for i in range(num_classes)]

    for class_idx in range(num_classes):
        indices = class_indices[class_idx]
        for _ in range(num_pairs_per_class):
            idx1, idx2 = np.random.choice(indices, size=2, replace=False)
            pairs.append([x[idx1], x[idx2]])
            labels.append(1)

            other_class_idx = np.random.choice(list(range(num_classes)), size=1)[0]
            while other_class_idx == class_idx:
                other_class_idx = np.random.choice(list(range(num_classes)), size=1)[0]

            idx3 = np.random.choice(class_indices[other_class_idx])
            pairs.append([x[idx1], x[idx3]])
            labels.append(0)

    return np.array(pairs), np.array(labels)

# Instantiate the Siamese network
input_shape = x_train.shape[1:]
siamese_model = create_siamese_network(input_shape)

# Compile the model
siamese_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Create pairs for training and validation with data augmentation
num_classes = len(np.unique(y_train))
train_pairs, train_labels = create_pairs_with_augmentation(x_train, y_train, num_classes, num_pairs_per_class=100)
test_pairs, test_labels = create_pairs_with_augmentation(x_test, y_test, num_classes, num_pairs_per_class=50)

# Custom training loop
batch_size = 128
epochs = 20
steps_per_epoch = len(train_pairs) // batch_size

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    epoch_loss = []
    epoch_accuracy = []

    # Shuffle training pairs each epoch
    indices = np.random.permutation(len(train_pairs))
    train_pairs_shuffled = train_pairs[indices]
    train_labels_shuffled = train_labels[indices]

    for step in range(steps_per_epoch):
        batch_pairs = train_pairs_shuffled[step * batch_size : (step + 1) * batch_size]
        batch_labels = train_labels_shuffled[step * batch_size : (step + 1) * batch_size]

        loss, accuracy = siamese_model.train_on_batch([batch_pairs[:, 0], batch_pairs[:, 1]], batch_labels)
        epoch_loss.append(loss)
        epoch_accuracy.append(accuracy)

    # Calculate average loss and accuracy for the epoch
    mean_loss = np.mean(epoch_loss)
    mean_accuracy = np.mean(epoch_accuracy)
    print(f"Mean Loss: {mean_loss:.4f}, Mean Accuracy: {mean_accuracy:.4f}")

# Evaluate the Siamese network on test pairs
test_loss, test_accuracy = siamese_model.evaluate([test_pairs[:, 0], test_pairs[:, 1]], test_labels)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 1/20
Mean Loss: 7.6564, Mean Accuracy: 0.4979
Epoch 2/20
Mean Loss: 7.6326, Mean Accuracy: 0.4995
Epoch 3/20
Mean Loss: 7.6087, Mean Accuracy: 0.5010
Epoch 4/20
Mean Loss: 7.5770, Mean Accuracy: 0.5031
Epoch 5/20
Mean Loss: 7.6405, Mean Accuracy: 0.4990
Epoch 6/20
Mean Loss: 7.5849, Mean Accuracy: 0.5026
Epoch 7/20
Mean Loss: 7.6643, Mean Accuracy: 0.4974
Epoch 8/20
Mean Loss: 7.6326, Mean Accuracy: 0.4995
Epoch 9/20
Mean Loss: 7.6167, Mean Accuracy: 0.5005
Epoch 10/20
Mean Loss: 7.5452, Mean Accuracy: 0.5052
Epoch 11/20
Mean Loss: 7.6405, Mean Accuracy: 0.4990
Epoch 12/20
Mean Loss: 7.6167, Mean Accuracy: 0.5005
Epoch 13/20
Mean Loss: 7.6008, Mean Accuracy: 0.5016
Epoch 14/20
Mean Loss: 7.6246, Mean Accuracy: 0.5000
Epoch 15/20
Mean Loss: 7.6167, Mean Accuracy: 0.5005
Epoch 16/20
Mean Loss: 7.6008, Mean Accuracy: 0.5016
Epoch 17/20
Mean Loss: 7.6643, Mean Accuracy: 0.4974
Epoch 18/20
Mean Loss: 7.6643, Mean Accuracy: 0.4974
Epoch 19/20
Mean Loss: 7.5690, Mean Accuracy: 0.5036
Ep